In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
import sys
sys.path.append('..')

import os

from utils import GCStorage
from constants import CREDENTIAL_PATH

In [3]:
storage = GCStorage('FX_Trading', 'integral_data', CREDENTIAL_PATH)

In [4]:
storage.list_files('raw_data')

({'raw_data': {'': {}, 'FX_Integral_data.zip': {}}},
 {'': {}, 'FX_Integral_data.zip': {}})

In [15]:
os.makedirs(os.environ['HOME'] + '/' + 'temp_store/raw_data', exist_ok=True)

storage.download( os.environ['HOME'] + '/' + 'temp_store/raw_data/FX_Integral_data.zip', 'raw_data/FX_Integral_data.zip',)

'raw_data/FX_Integral_data.zip downloaded from bucket.'

# Creat structure

In [4]:
import re
from collections import defaultdict
import pprint as pp

from tqdm import tqdm
RE_FNAME = re.compile(r'LP\-(?P<idx>[0-9])\-STRM\-[0-9]\-(?P<pair>[A-Z]+).*\.csv')

In [29]:
store = '/home/jingboyang/temp_store/raw_data/fx_integral_data'

total_files = 0

result_dict = defaultdict(list)

for folder in os.listdir(store):
    if '.zip' not in folder:
        for f in os.listdir(f'{store}/{folder}'):
            full_path = f'{store}/{folder}/{f}'
            
            total_files += 1
            
            result = RE_FNAME.search(f)
            gd = result.groupdict()
            
            lp_idx = gd['idx']
            pair = gd['pair']
            
            # print(pair, lp_idx)
            result_dict[pair].append((lp_idx, full_path))
            

print(total_files)

# pp.pprint(result_dict)
print(result_dict.keys())

1290
dict_keys(['EURUSD', 'NZDUSD', 'AUDUSD', 'GBPUSD', 'USDJPY', 'USDCAD', 'USDSEK', 'USDCHF'])


In [25]:
import pandas as pd

import shutil
import csv
import numpy as np
from subprocess import call

from dateutil.parser import parse
from pathlib import Path

In [12]:
raw_path = Path('/home/jingboyang/temp_store/raw_data/fx_integral_data')
new_path = Path('/home/jingboyang/temp_store/raw_data/organized_data')

new_path.mkdir(parents=True, exist_ok=True)

In [32]:
# for pair in result_dict:
#    print(pair)
for pair in ['USDCHF', 'USDSEK']:
    pair_path = new_path / pair
    
    organized = []
    
    for lp_idx, fpath in tqdm(result_dict[pair]):
        
        start_time_text = None
        with open(fpath, newline='') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                # print(row['first_name'], row['last_name'])
                start_time_text = row['time']
                break
        
        if start_time_text is not None:

            start_time = parse(start_time_text, dayfirst=False)
            file_date = start_time.replace(minute=0, hour=0, second=0, microsecond=0)
            # end_time = parse(df['time'][len(df) - 1], dayfirst=False)

            folder = fpath.split('/')[-2]
            fname = fpath.split('/')[-1]

            fsize = os.path.getsize(fpath)

            # organized.append((start_time, end_time, folder, fname))
            organized.append({'file_date': file_date, 'lp_idx': lp_idx, 'pair': pair,
                              'start_time': start_time, # 'end_date': end_time,
                              'folder': folder, 'fpath': fpath, 'fname': fname,
                              'data_length': fsize})
                # print(f"{df['time'][0]} {fpath[-80:]}")
            # except:
            #     print(f'Unable to process: {fpath}')

    
    organized = sorted(organized, key=lambda x: (x['file_date'], x['lp_idx'], x['data_length']))
    
    prev_date = None
    cur_path = None
    counter = 0
    print(pair)
    for item in organized:
        # print(f'{item["file_date"]}: {item["lp_idx"]}')
        nice_date = item['file_date'].strftime("%Y%m%d")
        if item['file_date'] != prev_date:
            print(f'{prev_date}: {counter}')
            cur_path = pair_path / nice_date
            cur_path.mkdir(parents=True, exist_ok=True)
            counter = 0

        counter += 1
        new_fname = f'{nice_date}-{item["pair"]}-{item["lp_idx"]}.csv'
        # shutil.copyfile(item['fpath'], cur_path / new_fname)
        
        call(['cp', str(item['fpath']), str(cur_path / new_fname)])

        prev_date = item['file_date']

100%|██████████| 160/160 [00:00<00:00, 6656.96it/s]


USDCHF
None: 0
2019-02-01 00:00:00: 15
2019-02-03 00:00:00: 5
2019-02-04 00:00:00: 5
2019-02-05 00:00:00: 5
2019-02-06 00:00:00: 5
2019-02-07 00:00:00: 5
2019-02-08 00:00:00: 5
2019-02-10 00:00:00: 5
2019-02-11 00:00:00: 5
2019-02-12 00:00:00: 5
2019-02-13 00:00:00: 5
2019-02-14 00:00:00: 5
2019-02-15 00:00:00: 5
2019-02-17 00:00:00: 5
2019-02-18 00:00:00: 5
2019-02-19 00:00:00: 5
2019-02-20 00:00:00: 5
2019-02-21 00:00:00: 5
2019-02-22 00:00:00: 5
2019-02-24 00:00:00: 5
2019-02-25 00:00:00: 5
2019-02-26 00:00:00: 5
2019-02-27 00:00:00: 5
2019-02-28 00:00:00: 5


100%|██████████| 170/170 [00:09<00:00, 17.52it/s]


USDSEK
None: 0
2019-01-07 00:00:00: 2
2019-01-08 00:00:00: 2
2019-01-09 00:00:00: 2
2019-01-10 00:00:00: 2
2019-01-11 00:00:00: 2
2019-02-01 00:00:00: 13
2019-02-03 00:00:00: 4
2019-02-04 00:00:00: 5
2019-02-05 00:00:00: 5
2019-02-06 00:00:00: 5
2019-02-07 00:00:00: 5
2019-02-08 00:00:00: 5
2019-02-10 00:00:00: 4
2019-02-11 00:00:00: 5
2019-02-12 00:00:00: 5
2019-02-13 00:00:00: 5
2019-02-14 00:00:00: 5
2019-02-15 00:00:00: 5
2019-02-17 00:00:00: 4
2019-02-18 00:00:00: 5
2019-02-19 00:00:00: 5
2019-02-20 00:00:00: 5
2019-02-21 00:00:00: 5
2019-02-22 00:00:00: 5
2019-02-24 00:00:00: 4
2019-02-25 00:00:00: 5
2019-02-26 00:00:00: 5
2019-02-27 00:00:00: 5
2019-02-28 00:00:00: 5


# Moving stuff back to storage

In [38]:
for pair in os.listdir(str(new_path)):
    print(pair)
    for date in tqdm(os.listdir(str(new_path / pair))):
        for fname in os.listdir(str(new_path / pair / date)):
            fpath = new_path / pair / date / fname

            cloud_path = Path('organized_data') / pair / date / fname

            storage.upload(fpath, cloud_path)

            # break
        # break
    # break

  0%|          | 0/25 [00:00<?, ?it/s]

NZDUSD


  0%|          | 0/30 [00:00<?, ?it/s]

USDSEK


  0%|          | 0/25 [00:00<?, ?it/s]

...

  0%|          | 0/25 [00:00<?, ?it/s]

USDJPY


  0%|          | 0/25 [00:00<?, ?it/s]

GBPUSD


100%|██████████| 25/25 [06:37<00:00, 15.91s/it]
